# LangGraph Persistence with Couchbase

### LangGraph

LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions. This tutorial focuses on showcasing persisting state of [LangGraph](https://github.com/langchain-ai/langgraph) with Couchbase.

### Checkpointer

Checkpointers in LangGraph save snapshots of graph state at each execution step, enabling memory between interactions, human-in-the-loop workflows, and fault tolerance. By organizing states into "threads" with unique IDs - `thread_id`, they preserve conversation history and allow time travel debugging. Checkpointers implement methods to store, retrieve, and list checkpoints, with various backend options (in-memory, Couchbase, SQLite) to suit different application needs. This persistence layer is what enables agents to maintain context across multiple user interactions and recover gracefully from failures.

### Couchbase as a Checkpointer

This tutorial focuses on implementing a LangGraph checkpointer with Couchbase, leveraging Couchbase's distributed architecture, JSON document model, and high availability to provide robust persistence for agent workflows. Couchbase's scalability and flexible query capabilities make it an ideal backend for managing complex conversation states across multiple users and sessions.

# How to use Couchbase Checkpointer



This tutorial focuses on using a LangGraph checkpointer with Couchbase using the dedicated [langgraph-checkpointer-couchbase](https://pypi.org/project/langgraph-checkpointer-couchbase/) package.

This package provides a seamless way to persist LangGraph agent states in Couchbase, enabling:

- State persistence across application restarts
- Retrieval of historical conversation steps
- Continued conversations from previous checkpoints
- Both synchronous and asynchronous interfaces

## Setup environment

Requires Couchbase Python SDK and langgraph package

In [1]:
%%capture --no-stderr
%pip install -U langgraph==0.3.22 langgraph-checkpointer-couchbase 

This particular example uses OpenAI's GPT 4.1-mini as the model

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

## Setup model and tools for the graph

We will be creating a [ReAct Agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) for this demo. Let's create a custom tool which our agent can call to get more information.

We are using a tool `get_weather` which gives the weather information based on the city. This tool gives weather information based on the city. We are also setting up the ChatGPT model here.

In [3]:
from typing import Literal
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

### Couchbase Connection and intialization

There are 2 ways to initialize a saver.

1. `from_conn_info` - Provide details of the connection string, username, password. The package will handle connection itself.
2. `from_cluster` - Provide a connected Couchbase.Cluster object. 

We will be using `from_conn_info` in the tutorial, but any of the above can be used as per requirements


## Use sync connection (CouchbaseSaver)

Below is usage of CouchbaseSaver (for synchronous use of graph, i.e. `.invoke()`, `.stream()`). CouchbaseSaver implements four methods that are required for any checkpointer:

- `.put` - Store a checkpoint with its configuration and metadata.
- `.put_writes` - Store intermediate writes linked to a checkpoint (i.e. pending writes).
- `.get_tuple` - Fetch a checkpoint tuple using a given configuration (`thread_id` and `checkpoint_id`).
- `.list` - List checkpoints that match a given configuration and filter criteria.

Here we will create a Couchbase connection. We are using local setup with bucket `test`, `langgraph` scope. You may change bucket and scope if required. We will also require `checkpoints` and `checkpoint_writes` as collections inside.

Then a [ReAct Agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) is created with GPT Model, weather tool and Couchbase checkpointer.

LangGraph's graph is invoked with message for GPT, storing all the state in Couchbase. We use get, get_tuple and list methods to fetch the states again

In [4]:
from langgraph_checkpointer_couchbase import CouchbaseSaver

with CouchbaseSaver.from_conn_info(
    cb_conn_str="couchbase://localhost",
    cb_username="Administrator",
    cb_password="password",
    bucket_name="test",
    scope_name="langgraph",
) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)
    
    latest_checkpoint = checkpointer.get(config)
    latest_checkpoint_tuple = checkpointer.get_tuple(config)
    checkpoint_tuples = list(checkpointer.list(config))

/Users/lokeshgoel/.pyenv/versions/langgraphCheckpointer/lib/python3.11/site-packages/couchbase/management/collections.py:123: CouchbaseDeprecationWarning: Method create_collection with signature (collection: couchbase.management.logic.collections_logic.CollectionSpec, *options: couchbase.management.options.CreateCollectionOptions, **kwargs: Dict[str, Any]) -> None is deprecated and will be removed in a future release. Use create_collection with signature (scope_name: str, collection_name: str, settings: Optional[couchbase.management.logic.collections_logic.CreateCollectionSettings], *options: couchbase.management.options.CreateCollectionOptions, **kwargs: Dict[str, Any]) -> None instead.
  Supportability.method_signature_deprecated(


In [5]:
latest_checkpoint

{'v': 2,
 'ts': '2025-04-21T05:56:20.853383+00:00',
 'id': '1f01e755-968d-620e-8003-459cce96ee2d',
 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='52a4e1f3-ba84-4b7f-9773-9e57ae7962a7'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZZEwpygrpyohAh698zDoxSbl', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-BOeQdk2ax6ZVqORyLD2mn5K3VUTby', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e25bd6cf-e898-4331-9774-91340896

In [6]:
latest_checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f01e755-968d-620e-8003-459cce96ee2d'}}, checkpoint={'v': 2, 'ts': '2025-04-21T05:56:20.853383+00:00', 'id': '1f01e755-968d-620e-8003-459cce96ee2d', 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='52a4e1f3-ba84-4b7f-9773-9e57ae7962a7'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZZEwpygrpyohAh698zDoxSbl', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be

In [7]:
checkpoint_tuples

[CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f01e755-968d-620e-8003-459cce96ee2d'}}, checkpoint={'v': 2, 'ts': '2025-04-21T05:56:20.853383+00:00', 'id': '1f01e755-968d-620e-8003-459cce96ee2d', 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='52a4e1f3-ba84-4b7f-9773-9e57ae7962a7'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZZEwpygrpyohAh698zDoxSbl', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79b

## Use async connection (AsyncCouchbaseSaver)

Below is a reference usage of AsyncCouchbaseSaver (for asynchronous use of graph, i.e. `.ainvoke()`, `.astream()`). AsyncCouchbaseSaver implements four methods that are required for any async checkpointer:

- `.aput` - Store a checkpoint with its configuration and metadata.
- `.aput_writes` - Store intermediate writes linked to a checkpoint (i.e. pending writes).
- `.aget_tuple` - Fetch a checkpoint tuple using a given configuration (`thread_id` and `checkpoint_id`).
- `.alist` - List checkpoints that match a given configuration and filter criteria.

This is the asynchronous example, Here we will create a Couchbase connection. We are using local setup with bucket `test`, `langgraph` scope. You may change bucket and scope if required. We will also require `checkpoints` and `checkpoint_writes` as collections inside.

Then a [ReAct Agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) is created with GPT Model, weather tool and Couchbase checkpointer.

LangGraph's graph is invoked with message for GPT, storing all the state in Couchbase. We use get, get_tuple and list methods to fetch the states again

In [8]:
from langgraph_checkpointer_couchbase import AsyncCouchbaseSaver

async with AsyncCouchbaseSaver.from_conn_info(
    cb_conn_str="couchbase://localhost",
    cb_username="Administrator",
    cb_password="password",
    bucket_name="test",
    scope_name="langgraph",
) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "2"}}
    res = await graph.ainvoke(
        {"messages": [("human", "what's the weather in nyc")]}, config
    )

    latest_checkpoint = await checkpointer.aget(config)
    latest_checkpoint_tuple = await checkpointer.aget_tuple(config)
    checkpoint_tuples = [c async for c in checkpointer.alist(config)]

SELECT * FROM test.langgraph.`checkpoints` WHERE thread_id = $1 AND checkpoint_ns = $2 ORDER BY checkpoint_id DESC LIMIT 1
SELECT * FROM test.langgraph.`checkpoints` WHERE thread_id = $1 AND checkpoint_ns = $2 ORDER BY checkpoint_id DESC LIMIT 1
SELECT * FROM test.langgraph.`checkpoints` WHERE thread_id = $1 AND checkpoint_ns = $2 ORDER BY checkpoint_id DESC LIMIT 1


In [9]:
latest_checkpoint

{'v': 2,
 'ts': '2025-04-21T05:56:24.214460+00:00',
 'id': '1f01e755-b69a-6e66-8003-0c1653e6f222',
 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='d15a78fc-4527-40e4-97ed-d8592c5b0f0a'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UvS4kYUWsRUnAFl8dkR2PNHW', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79be41f', 'id': 'chatcmpl-BOeQfGu6yn2xfKKJIUP8dql8sAeh4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-80fc714b-9692-4b2c-b77a-4dad4e

In [10]:
latest_checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f01e755-b69a-6e66-8003-0c1653e6f222'}}, checkpoint={'v': 2, 'ts': '2025-04-21T05:56:24.214460+00:00', 'id': '1f01e755-b69a-6e66-8003-0c1653e6f222', 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='d15a78fc-4527-40e4-97ed-d8592c5b0f0a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UvS4kYUWsRUnAFl8dkR2PNHW', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b79

In [11]:
checkpoint_tuples

[CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f01e755-b69a-6e66-8003-0c1653e6f222'}}, checkpoint={'v': 2, 'ts': '2025-04-21T05:56:24.214460+00:00', 'id': '1f01e755-b69a-6e66-8003-0c1653e6f222', 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='d15a78fc-4527-40e4-97ed-d8592c5b0f0a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UvS4kYUWsRUnAFl8dkR2PNHW', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_79b7